<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/mary_ft_and_context_playground_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Google colab notebook demonstrates the following:
* fine tuning on a small custom QA dataset
* experiment with use of dataset 'context' at prompt time

# Get access to Gemma via your Kaggle account:
* Log into your Kaggle account
* Request access to Gemma models using your Kaggle account. You can follow these instructions here: https://www.kaggle.com/code/nilaychauhan/get-started-with-gemma-using-kerasnlp
* You need to wait for confirmation. Note that this didn't take too long for me.
* Create an API key in your Kaggle account you will need later. You can follow these instructions here: https://christianjmills.com/posts/kaggle-obtain-api-key-tutorial/

# Ensure your Colab notebook can access Gemma:
* Add the Kaggle API key into your COLAB secrets. You can follow these instructions here: https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0

# Select an AI hardware accelerator
* Select hardware options near the top right of your Colab notebook
* I tested with A100 and it worked well. Note that I have a Colab Pro subscription.

# Install required packages

In [1]:
%%time
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3.3.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
CPU times: user 48.2 ms, sys: 7.35 ms, total: 55.5 ms
Wall time: 7.83 s


# Import required packages

In [2]:
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
from keras import backend as K
import tensorflow
from IPython.display import Markdown, display
import textwrap
from google.colab import userdata
import json
import pandas as pd
import gc
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil
from google.colab import drive

# Configure this notebook

In [3]:
# set up Keras parameters recommended by Google
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00" # Avoid memory fragmentation on JAX backend.

# integrate Kaggle API
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME') # Link to KAGGLE API secret key
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY') # Link to KAGGLE API secret key

# Define useful functions

In [4]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

# Load/explore the dataset
* note I got this by running Mary's original notebook and copying the unpacked input folder

In [5]:
DATASET_PATH='/content/drive/MyDrive/Kaggle_X/Mary_ESSA/input/attempt-930/ESSA qna_csv.csv'
if not os.path.exists(DATASET_PATH):
  raise Exception("Cannot find the dataset")
df = pd.read_csv(DATASET_PATH)
pd.set_option('display.max_colwidth', None)
df.describe()

,Question,Answer
count,101,101
unique,95,99
top,What is the state's role in ESSA?,"Under ESSA, states get to decide the education plans for their schools within a framework provided by the federal government. The law also offers parents a chance to weigh in on these plans. This is important if your child gets special education services."
freq,2,2


In [6]:
df.head(5)

,Question,Answer
0,Does my state still have to test 95 percent of its students?,"ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments."
1,How do the students (up to 1 percent) who receive the alternate assessment count in terms of the state’s 95 percent requirement?,"As long as they meet the other requirements around alternate assessments (e.g. alignment with the state’s standards), states may count students who are assessed based on alternate academic achievement standards for purposes of meeting the 95 percent participation rate."
2,What are the related mandates or prohibitions related to Common Core?,"While states must maintain “challenging academic standards” (floor set as: at least three achievement levels in math, English/language arts, and science), there is a strong prohibition on the federal government using any of its authority to mandate or incentivize the use of particular standards. This prohibition not only applies to standards, but also assessments, curriculum, etc. The bill does note, however, that nothing in the law prohibits states from voluntarily entering into partnerships on standards."
3,What kind of alignment is required between elementary and secondary standards and higher education?,"ESSA requires that states demonstrate that their challenging academic standards are aligned with entrance requirements for public institutions of higher educations (IHEs) within that state. However, the legislation was also clear that this does permit the state’s IHEs to set or determine the state’s standards."
4,Are states required to submit their standards for approval by the U.S. Department of Education?,"No. There is clear language in the bill that no state shall be required to submit its standards to the federal government for review or approval. (Standards underlie the accountability system, which is part of the state Title I plan submitted to the Department.) Again, states must maintain challenging academic standards, but the law is very clear that states are not required to seek federal approval of their standards and can make changes to them without federal approval."


# Load the Gemma 2B base model

In [7]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

CPU times: user 9.45 s, sys: 9.5 s, total: 18.9 s
Wall time: 45.4 s


# Ask the non-fined-tuned, non-contexted model a question from the dataset

In [8]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 1min 1s, sys: 1.01 s, total: 1min 2s
Wall time: 29.6 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The Every Student Succeeds Act (ESSA) is a federal law that replaced the No Child Left Behind Act (NCLB) in 2015. ESSA is a reauthorization of the Elementary and Secondary Education Act (ESEA), which was passed in 1965. ESSA is a comprehensive education law that aims to improve the quality of education for all students, regardless of their background or ability.<br><br>The law requires states to develop and implement plans to improve student achievement and to ensure that all students are prepared for success in college, career, and life. ESSA also requires states to collect and report data on student achievement and progress, and to use this data to inform decision-making about education policy and practice.<br><br>The law also includes a number of related mandates and prohibitions. For example, ESSA requires states to provide parents with information about their child’s progress in school, and to allow parents to participate in decisions about their child’s education. It also prohibits states from using federal funds to discriminate against students based on their race, color, national origin, or disability.<br><br>In addition, ESSA includes a number of prohibitions related to Common Core. Common Core is a set of standards for math and English language arts that was developed by the National Governors Association and the Council of Chief State School Officers. The standards were adopted by many states, but they were controversial because they were seen as too restrictive and difficult to implement.<br><br>As a result, many states have adopted their own standards, and some have even repealed Common Core altogether. ESSA prohibits states from adopting or implementing Common Core, and it also prohibits states from using federal funds to support Common Core.<br><br>In conclusion, ESSA is a comprehensive education law that aims to improve the quality of education for all students. It includes a number of related mandates and prohibitions, including those related to Common Core.</blockquote></font>

# Let's try a harder one

In [9]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 9.63 s, sys: 5.67 ms, total: 9.64 s
Wall time: 9.6 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, your state still has to test 95 percent of its students.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). The purpose of ESSA is to ensure that all students have access to a high-quality education.<br><br>Question:<br>What are some of the requirements of ESSA?<br><br>Answer:<br>Some of the requirements of ESSA include:<br><br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education</blockquote></font>

# Analysis
* It seems to have gotten the answer right but
* Did it "hallucinate" and just get lucky?
* Also, the response format isn't well-formed

# Another hard one

In [10]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)



CPU times: user 9.66 s, sys: 13.8 ms, total: 9.67 s
Wall time: 9.63 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that provides states with more flexibility in how they implement the No Child Left Behind Act (NCLB). ESSA also requires states to develop and implement plans to improve student achievement and close achievement gaps.<br><br>Question:<br>What is the difference between a state plan and a local plan?<br><br>Answer:<br>A state plan is a document that outlines how a state will implement ESSA. A local plan is a document that outlines how a school district or other local education agency will implement ESSA.<br><br>Question:<br>What are the requirements for a state plan?<br><br>Answer:<br>A state plan must include the following:<br>1. A description of how the state will implement ESSA.<br>2. A description of how the state will use data to improve student achievement and close achievement gaps.<br>3. A description of how the state will use Title I funds to support student achievement and close achievement gaps.<br>4. A description of how the state will use Title II funds to support teacher and principal effectiveness.<br>5. A description of how the state will use Title III funds to support English learners.<br>6. A description of how the state will use Title IV funds to support student success.<br>7. A description of how the state will use Title V funds to support student success.<br>8. A description of how the state will use Title VI funds to support student success.<br>9. A description of how the state will use Title VII funds to support student success.<br>10. A description of how the state will use Title VIII funds to support student success.<br>11. A description of how the state will use Title IX funds to support student success.<br>12. A description of how the state will use Title X funds to support student success.<br>13. A description of how the state will use Title XI funds to support student success.<br>14. A description of how the state will use Title XII funds to support student success.<br>15. A description of how the state will use Title XIII funds to support student success.<br>16. A description of how the state will use Title XIV funds to support student success.<br>17. A description of how the state will use Title XV funds to support student success.<br>18. A description of how the state will use Title XVI funds to support student success.<br>19. A description of how the state will use Title XVII funds to support student success.<br>20. A description of how the state will use Title XVIII funds to support student success.<br>21. A description of how the state will use Title XIX funds to support student success.<br>22. A description of how the state will use Title XX funds to support student success.<br>23. A description of how the state will use Title XXI funds to support student success.<br>24. A description of how the state will use Title XXII funds to support student success.<br>25. A description of how the state will use Title XXIII funds to support student success.<br>26. A description of how the state will use Title XXIV funds to support student success.<br>27. A description of how the state will use Title XXV funds to support student success.<br>28. A description of how the state will use Title XXVI funds to support student success.<br>29. A description of how the state will use Title XXVII funds to support student success.<br>30. A description of how the state will use Title XXVIII funds to support student success.<br>31. A description of how the state will use Title XXIX funds to support student success.<br>32. A description of how the state will use Title XXX funds to support student success.<br>33. A description of how the state will use Title XXXI funds to support student success.<br>34. A description of how the state will use Title XXXII funds to support student success.<br>35. A description of how the state will use Title XXXIII funds to support student success.<br>36. A description of how the state will use Title XXXIV funds to support student success.<br>37. A description of how the state will use Title XXXV funds to support student success.<br>38. A description of how the state will use Title XXXVI funds to support student success.<br>39. A description of how the state will use Title XXXVII funds to support student success.<br>40. A description of how the state will use Title XXXVIII funds to support student success</blockquote></font>

# Analysis of queries on Gemma 2B base model
* It seems to have gotten the answer wrong, has outdated information, or the information in the dataset should be considered the ground-truth fact.
* If it "hallucinated" the fact, it sounded really confident.
* Also, the response format isn't concise and it rambles

# Prepare dataset for fine-tuning

In [7]:
df.dropna(inplace=True)
print(df.shape)

template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
pre = '''The following is an excerpt from a conversation of a user with an AI assistant. '''\
      '''The assistant that can answer questions about ESSA. '''\
      '''ESSA stands for the Every Student Succeeds Act.'''

# format each training string, put them all into a list
ft_all_data = []
for idx, row in df.iterrows():
  ft_item = template.format(pre=pre, question=row['Question'], answer=row['Answer'])
  ft_all_data.append(ft_item)

# double-check
print("----")
print(ft_all_data[0])
print("----")
print(ft_all_data[1])
print("----")
print(ft_all_data[2])
print("----")
print(ft_all_data[-1])


(101, 2)
----
The following is an excerpt from a conversation of a user with an AI assistant. The assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.

Question:
Does my state still have to test 95 percent of its students? 

Answer:
ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments. 
----
The following is an excerpt from a conversation of a user with an AI assistant. The assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.

Question:
How do the students (up to 1 percent) who receive the alternate assessment count in terms of the state’s 95 percent requirement? 

Answer:
As long 

# Fine-tune the Gemma 2B base model

In [12]:
%%time

# Configure LORA rank=4 as fine-tuning method
gemma_lm.backbone.enable_lora(rank=4)

# Limit the input sequence length to X (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1024
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(ft_all_data, epochs=1, batch_size=1)

101/101 ━━━━━━━━━━━━━━━━━━━━ 61s 121ms/step - loss: 0.3033 - sparse_categorical_accuracy: 0.5006
CPU times: user 1min 24s, sys: 2.56 s, total: 1min 27s
Wall time: 1min 1s


# Query the fine-tuned model

In [13]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 28.9 s, sys: 562 ms, total: 29.4 s
Wall time: 30.2 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>ESSA requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of ESSA?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts must follow when teaching math and reading</blockquote></font>

In [14]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 10.7 s, sys: 2.77 ms, total: 10.7 s
Wall time: 10.7 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, your state still has to test 95 percent of its students.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose</blockquote></font>

In [15]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 10.6 s, sys: 4.64 ms, total: 10.6 s
Wall time: 10.6 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that provides states with more flexibility to design their own education systems. The law requires states to develop and implement plans to improve student achievement and to ensure that all students are prepared for success in college and career.<br><br>Question:<br>What are the requirements for states to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br>States must submit their standards for approval by the U.S. Department of Education within 18 months of the law’s enactment. The standards must be based on the state’s educational goals and objectives, and must include a description of how the state will measure student achievement.<br><br>Question:<br>What are the consequences for states that do not submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br>States that do not submit their standards for approval by the U.S. Department of Education may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for students who do not meet the standards set by their state?<br><br>Answer:<br>Students who do not meet the standards set by their state may be subject to disciplinary action, including suspension or expulsion.<br><br>Question:<br>What are the consequences for teachers who do not meet the standards set by their state?<br><br>Answer:<br>Teachers who do not meet the standards set by their state may be subject to disciplinary action, including termination.<br><br>Question:<br>What are the consequences for schools that do not meet the standards set by their state?<br><br>Answer:<br>Schools that do not meet the standards set by their state may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for districts that do not meet the standards set by their state?<br><br>Answer:<br>Districts that do not meet the standards set by their state may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for states that do not submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br>States that do not submit their standards for approval by the U.S. Department of Education may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for students who do not meet the standards set by their state?<br><br>Answer:<br>Students who do not meet the standards set by their state may be subject to disciplinary action, including suspension or expulsion.<br><br>Question:<br>What are the consequences for teachers who do not meet the standards set by their state?<br><br>Answer:<br>Teachers who do not meet the standards set by their state may be subject to disciplinary action, including termination.<br><br>Question:<br>What are the consequences for schools that do not meet the standards set by their state?<br><br>Answer:<br>Schools that do not meet the standards set by their state may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for districts that do not meet the standards set by their state?<br><br>Answer:<br>Districts that do not meet the standards set by their state may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for states that do not submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br>States that do not submit their standards for approval by the U.S. Department of Education may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for students who do not meet the standards set by their state?<br><br>Answer:<br>Students who do not meet the standards set by their state may be subject to disciplinary action, including suspension or expulsion.<br><br>Question:<br>What are the consequences for teachers who do not meet the standards set by their state?<br><br>Answer:<br>Teachers who do not meet the standards set by their state may be subject to disciplinary action, including termination.<br><br>Question:<br>What are the consequences for schools that do not meet the standards set by their state?<br><br>Answer:<br>Schools that do not meet the standards set by their state may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for districts that do not meet the standards set by their state?<br><br>Answer:<br>Districts that do not meet the standards set by their state may be subject to sanctions, including the loss of federal funding.<br><br>Question:<br>What are the consequences for states that do not submit their standards for approval by the U.</blockquote></font>

# Analysis of queries on fined-tuned Gemma 2B base model
* The results on the fine-tuned model aren't great both in terms of factual correctness and conciseness of response
* Possible issues/resolutions
  * fine-tuning dataset might be too small
  * try different fine-tuning parameters
  * try the Gemma instruction tuned version

# Load the Gemma 2B instruction tuned model

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

100%|██████████| 554/554 [00:00<00:00, 706kB/s]


100%|██████████| 4.67G/4.67G [05:12<00:00, 16.0MB/s]


100%|██████████| 401/401 [00:00<00:00, 541kB/s]


100%|██████████| 4.04M/4.04M [00:01<00:00, 2.30MB/s]


# Try the various prompts on the instruction tuned Gemma 2B model

In [9]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 58.4 s, sys: 855 ms, total: 59.3 s
Wall time: 26.5 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>As an AI assistant, I am unable to provide specific information about the Common Core State Standards Act (ESSA). For the most up-to-date and accurate information about ESSA mandates and prohibitions related to Common Core, please refer to the official website of the U.S. Department of Education.</blockquote></font>

In [10]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 705 ms, sys: 70 µs, total: 706 ms
Wall time: 704 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>I am unable to provide specific information about your state's ESSA testing requirements, as I do not have access to real-time data. For the most up-to-date information on ESSA testing in your state, please check with your state education agency or the U.S. Department of Education website.</blockquote></font>

In [11]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 378 ms, sys: 224 µs, total: 378 ms
Wall time: 377 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education under the Every Student Succeeds Act (ESSA).</blockquote></font>

# Analysis of queries on instruction tuned Gemma 2B model
* It seems the instruction tuned model seems best so far
* The responses are concise
* The model also says it does not know when that's the case
* It seems the last question is still problematic, either because the model has old information, it indeed "hallucinated", or the ESSA dataset has the ground-truth fact

# Let's try to answer the questions with the "correct" context in the prompt

In [12]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                             ''' ESSA stands for the Every Student Succeeds Act.'''
                             ''' Use the following context to answer the question below.''',
                         context='''While states must maintain “challenging academic standards” '''
                                 '''(floor set as: at least three achievement levels in math, English/language arts, '''
                                 '''and science), there is a strong prohibition on the federal government using any '''
                                 '''of its authority to mandate or incentivize the use of particular standards. '''
                                 '''This prohibition not only applies to standards, but also assessments, '''
                                 '''curriculum, etc. The bill does note, however, that nothing in the law '''
                                 '''prohibits states from voluntarily entering into partnerships on standards.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 308 ms, sys: 0 ns, total: 308 ms
Wall time: 307 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>While states must maintain “challenging academic standards” (floor set as: at least three achievement levels in math, English/language arts, and science), there is a strong prohibition on the federal government using any of its authority to mandate or incentivize the use of particular standards. This prohibition not only applies to standards, but also assessments, curriculum, etc. The bill does note, however, that nothing in the law prohibits states from voluntarily entering into partnerships on standards.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The context does not provide any information about Common Core, so I cannot answer this question from the provided context.</blockquote></font>

In [13]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''ESSA requires that a state’s accountability system must measure the performance of '''
                                 '''95 percent of students by looking at a variety of indicators. One of the indicators '''
                                 '''is “academic achievement as measured by proficiency on the annual assessments.” '''
                                 '''For this reason, in order to measure the overall achievement of 95 percent of students, '''
                                 '''95 percent must take the annual assessments.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 396 ms, sys: 0 ns, total: 396 ms
Wall time: 395 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The passage does not specify whether or not your state still has to test 95 percent of its students, so I cannot answer this question from the context.</blockquote></font>

In [14]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''There is clear language in the bill that no state shall be required to submit its standards '''
                                  '''to the federal government for review or approval. (Standards underlie the accountability system, '''
                                  '''which is part of the state Title I plan submitted to the Department.) '''
                                  '''Again, states must maintain challenging academic standards, but the law is '''
                                  '''very clear that states are not required to seek federal approval of their standards '''
                                  '''and can make changes to them without federal approval.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 435 ms, sys: 0 ns, total: 435 ms
Wall time: 434 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>There is clear language in the bill that no state shall be required to submit its standards to the federal government for review or approval. (Standards underlie the accountability system, which is part of the state Title I plan submitted to the Department.) Again, states must maintain challenging academic standards, but the law is very clear that states are not required to seek federal approval of their standards and can make changes to them without federal approval.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The passage does not specify whether states are required to submit their standards for approval by the U.S. Department of Education, so I cannot answer this question from the provided context.</blockquote></font>

# Let's fine-tune the instruction tuned Gemma 2B

In [15]:
%%time

# Configure LORA rank=4 as fine-tuning method
gemma_lm.backbone.enable_lora(rank=4)

# Limit the input sequence length to X (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1024
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(ft_all_data, epochs=1, batch_size=1)

101/101 ━━━━━━━━━━━━━━━━━━━━ 60s 120ms/step - loss: 0.4492 - sparse_categorical_accuracy: 0.4655
CPU times: user 1min 23s, sys: 2.35 s, total: 1min 25s
Wall time: 1min


# Try the queries without context on the fine-tuned version of the instruction tuned Gemma 2B model

In [16]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 19.8 s, sys: 524 ms, total: 20.4 s
Wall time: 21.3 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Sure, here's a summary of the related mandates and prohibitions related to Common Core:<br><br>**Mandates:**<br><br>* Schools must align their curriculum with the Common Core State Standards (CCSS).<br>* Schools must provide students with access to high-quality instructional materials aligned with the CCSS.<br>* Schools must use the results of state assessments to inform instruction and accountability measures.<br>* Schools must provide parents and families with information about their child's progress and the CCSS.<br><br>**Prohibitions:**<br><br>* Schools cannot discriminate against students based on race, ethnicity, or socioeconomic status.<br>* Schools cannot use standardized test scores to make decisions about students' education.<br>* Schools cannot use the CCSS as a means to control students or teachers.<br>* Schools cannot use the CCSS to impose punitive consequences on students.</blockquote></font>

In [17]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 363 ms, sys: 1.77 ms, total: 365 ms
Wall time: 363 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>No, the Every Student Succeeds Act (ESSA) does not require states to test 95 percent of their students.</blockquote></font>

In [18]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 417 ms, sys: 519 µs, total: 417 ms
Wall time: 416 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education under the Every Student Succeeds Act (ESSA).</blockquote></font>

# Analysis of queries (without context) on fine-tuned version of instruction tuned Gemma 2B

# Let's try to answer the questions with the "correct" context in the prompt, on the fine-tuned instruction tuned Gemma 2B


In [19]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                             ''' ESSA stands for the Every Student Succeeds Act.'''
                             ''' Use the following context to answer the question below.''',
                         context='''While states must maintain “challenging academic standards” '''
                                 '''(floor set as: at least three achievement levels in math, English/language arts, '''
                                 '''and science), there is a strong prohibition on the federal government using any '''
                                 '''of its authority to mandate or incentivize the use of particular standards. '''
                                 '''This prohibition not only applies to standards, but also assessments, '''
                                 '''curriculum, etc. The bill does note, however, that nothing in the law '''
                                 '''prohibits states from voluntarily entering into partnerships on standards.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 390 ms, sys: 0 ns, total: 390 ms
Wall time: 389 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>While states must maintain “challenging academic standards” (floor set as: at least three achievement levels in math, English/language arts, and science), there is a strong prohibition on the federal government using any of its authority to mandate or incentivize the use of particular standards. This prohibition not only applies to standards, but also assessments, curriculum, etc. The bill does note, however, that nothing in the law prohibits states from voluntarily entering into partnerships on standards.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The context does not provide any information about related mandates or prohibitions related to Common Core, so I cannot answer this question from the context.</blockquote></font>

In [20]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''ESSA requires that a state’s accountability system must measure the performance of '''
                                 '''95 percent of students by looking at a variety of indicators. One of the indicators '''
                                 '''is “academic achievement as measured by proficiency on the annual assessments.” '''
                                 '''For this reason, in order to measure the overall achievement of 95 percent of students, '''
                                 '''95 percent must take the annual assessments.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 444 ms, sys: 0 ns, total: 444 ms
Wall time: 442 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The passage does not specify whether or not your state still has to test 95 percent of its students, so I cannot answer this question from the context.</blockquote></font>

In [21]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''There is clear language in the bill that no state shall be required to submit its standards '''
                                  '''to the federal government for review or approval. (Standards underlie the accountability system, '''
                                  '''which is part of the state Title I plan submitted to the Department.) '''
                                  '''Again, states must maintain challenging academic standards, but the law is '''
                                  '''very clear that states are not required to seek federal approval of their standards '''
                                  '''and can make changes to them without federal approval.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 471 ms, sys: 576 µs, total: 471 ms
Wall time: 470 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>There is clear language in the bill that no state shall be required to submit its standards to the federal government for review or approval. (Standards underlie the accountability system, which is part of the state Title I plan submitted to the Department.) Again, states must maintain challenging academic standards, but the law is very clear that states are not required to seek federal approval of their standards and can make changes to them without federal approval.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The context does not specify whether states are required to submit their standards for approval by the U.S. Department of Education, so I cannot answer this question from the context.</blockquote></font>

# Analysis of queries with "correct" context on fine-tuned version of instruction tuned Gemma 2B

# Lets ask the same queries with context that clearly states the answer (on the fine-tuned instruction tuned Gemma 2B)

In [23]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''There are no related mandates or prohibitions related to Common Core.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 383 ms, sys: 0 ns, total: 383 ms
Wall time: 382 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>There are no related mandates or prohibitions related to Common Core.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The context does not provide any information about related mandates or prohibitions related to Common Core, so I cannot answer this question from the context.</blockquote></font>

In [24]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''Your state will have to test 95 percent of its students.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 411 ms, sys: 579 µs, total: 412 ms
Wall time: 410 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>Your state will have to test 95 percent of its students.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The context does not specify whether your state still has to test 95 percent of its students, so I cannot answer this question from the context.</blockquote></font>

In [25]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''States are not required to submit their standards for approval by the U.S. Department of Education.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 323 ms, sys: 0 ns, total: 323 ms
Wall time: 322 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>States are not required to submit their standards for approval by the U.S. Department of Education.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>No, states are not required to submit their standards for approval by the U.S. Department of Education.</blockquote></font>

# Let's now try Gemma 2 base

In [7]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")

CPU times: user 10.6 s, sys: 9.77 s, total: 20.3 s
Wall time: 57.6 s


In [8]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2min 10s, sys: 1.47 s, total: 2min 12s
Wall time: 1min 1s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The Every Student Succeeds Act (ESSA) is a federal law that replaced the No Child Left Behind Act (NCLB). ESSA requires states to develop and implement state academic standards and assessments, and to use these standards and assessments to hold schools and districts accountable for student achievement. ESSA also requires states to provide all students with a high-quality education, including those with disabilities, English learners, and students from low-income families.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge they need to be successful in college and career.<br><br>Question:<br>What are the related mandates or prohibitions related to the Common Core?<br><br>Answer:<br>The Common Core State Standards (CCSS) are a set of academic standards that were developed by a group of states in 2010. The CCSS are designed to provide a common set of standards for all states to use in their K-12 education systems. The CCSS are intended to help students develop the skills and knowledge</blockquote></font>

In [9]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 12.1 s, sys: 9.06 ms, total: 12.1 s
Wall time: 12.1 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, your state still has to test 95 percent of its students.<br><br>Question:<br>What is the ESSA?<br><br>Answer:<br>The ESSA is a federal law that replaced the No Child Left Behind Act. It requires states to set academic standards and test students in grades 3-8 and once in high school.<br><br>Question:<br>What are the ESSA’s goals?<br><br>Answer:<br>The ESSA’s goals are to improve student achievement, close achievement gaps, and provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements?<br><br>Answer:<br>The ESSA requires states to set academic standards and test students in grades 3-8 and once in high school. It also requires states to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for states?<br><br>Answer:<br>The ESSA requires states to set academic standards and test students in grades 3-8 and once in high school. It also requires states to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for schools?<br><br>Answer:<br>The ESSA requires schools to set academic standards and test students in grades 3-8 and once in high school. It also requires schools to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for teachers?<br><br>Answer:<br>The ESSA requires teachers to set academic standards and test students in grades 3-8 and once in high school. It also requires teachers to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for parents?<br><br>Answer:<br>The ESSA requires parents to set academic standards and test students in grades 3-8 and once in high school. It also requires parents to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for students?<br><br>Answer:<br>The ESSA requires students to set academic standards and test students in grades 3-8 and once in high school. It also requires students to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for schools?<br><br>Answer:<br>The ESSA requires schools to set academic standards and test students in grades 3-8 and once in high school. It also requires schools to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for teachers?<br><br>Answer:<br>The ESSA requires teachers to set academic standards and test students in grades 3-8 and once in high school. It also requires teachers to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for parents?<br><br>Answer:<br>The ESSA requires parents to set academic standards and test students in grades 3-8 and once in high school. It also requires parents to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for students?<br><br>Answer:<br>The ESSA requires students to set academic standards and test students in grades 3-8 and once in high school. It also requires students to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for schools?<br><br>Answer:<br>The ESSA requires schools to set academic standards and test students in grades 3-8 and once in high school. It also requires schools to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for teachers?<br><br>Answer:<br>The ESSA requires teachers to set academic standards and test students in grades 3-8 and once in high school. It also requires teachers to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for parents?<br><br>Answer:<br>The ESSA requires parents to set academic standards and test students in grades 3-8 and once in high school. It also requires parents to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for students?<br><br>Answer:<br>The ESSA requires students to set academic standards and test students in grades 3-8 and once in high school. It also requires students to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements for schools?<br><br>Answer:<br>The ESSA requires schools to set academic standards and test students in grades 3-8 and once in high school. It also requires schools to provide high-quality education for all students.<br><br>Question:<br>What are the ESSA’s requirements</blockquote></font>

In [10]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 12.1 s, sys: 7.33 ms, total: 12.1 s
Wall time: 12 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education.<br><br>Question:<br>What is the purpose of the ESSA?<br><br>Answer:<br>The ESSA is a federal law that aims to improve education for all students by providing states with more flexibility in how they design and implement their education systems.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question:<br>What are the key provisions of the ESSA?<br><br>Answer:<br>The ESSA provides states with more flexibility in how they design and implement their education systems. It also requires states to develop a plan for improving student achievement and closing achievement gaps.<br><br>Question</blockquote></font>

# Now let's try Gemma 2 2B instruction tuned

In [7]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

CPU times: user 10.5 s, sys: 9.73 s, total: 20.2 s
Wall time: 53.8 s


In [8]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2min 3s, sys: 1.64 s, total: 2min 5s
Wall time: 53.1 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The Every Student Succeeds Act (ESSA) does not directly address Common Core State Standards. <br><br>**Explanation:**<br><br>* **ESSA's Focus:** ESSA focuses on providing flexibility and support to states in designing their own education systems. It emphasizes accountability, but it doesn't dictate specific curriculum standards.<br>* **Common Core's Origins:** Common Core State Standards were developed by a group of state education officials and were adopted by many states. They were intended to provide a consistent set of standards for K-12 education.<br>* **ESSA's Role:** ESSA allows states to choose their own standards, including those aligned with Common Core. It also provides funding and support to states to implement these standards.<br><br>**Key Points:**<br><br>* ESSA does not mandate or prohibit Common Core.<br>* States have the authority to adopt or reject Common Core standards.<br>* ESSA provides resources and flexibility to states in implementing their chosen standards.<br><br><br>Let me know if you have any other questions about ESSA! <br><end_of_turn></blockquote></font>

In [9]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2.07 s, sys: 3.2 ms, total: 2.08 s
Wall time: 2.06 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>No, the Every Student Succeeds Act (ESSA) does not require states to test 95 percent of their students. <br><br>Here's why:<br><br>* **ESSA gives states flexibility:**  ESSA allows states to determine the appropriate level of testing for their students. <br>* **Focus on student needs:**  ESSA emphasizes using assessments to help students learn and improve, rather than just measuring performance. <br>* **State-specific requirements:**  States are responsible for setting their own testing requirements, as long as they meet the requirements of the law.<br><br>**Important Note:** While ESSA doesn't mandate a specific percentage of testing, states are still required to collect data on student performance. This data can be used to inform decisions about school improvement and student support. <br><end_of_turn></blockquote></font>

In [10]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2.58 s, sys: 954 µs, total: 2.59 s
Wall time: 2.57 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>No, states are not required to submit their standards for approval by the U.S. Department of Education under ESSA. <br><br>**Explanation:**<br><br>The Every Student Succeeds Act (ESSA) shifted the responsibility for setting educational standards from the federal government to the states. This means that states now have more autonomy in determining what their students should learn. <br><br>While ESSA does not require states to submit their standards for approval, it does provide a framework for states to develop and implement their own standards. <br><br>Here are some key points to remember:<br><br>* **States have the primary responsibility for setting educational standards.**<br>* **ESSA provides guidance and resources to help states develop their standards.**<br>* **States are encouraged to align their standards with the Common Core State Standards, but are not required to do so.**<br>* **States must ensure that their standards are rigorous and meet the needs of all students.**<br><br><br>Let me know if you have any other questions about ESSA! <br><end_of_turn></blockquote></font>

# Let's try Gemma 2 2B instruction tuned how with "correct" context

In [11]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                             ''' ESSA stands for the Every Student Succeeds Act.'''
                             ''' Use the following context to answer the question below.''',
                         context='''While states must maintain “challenging academic standards” '''
                                 '''(floor set as: at least three achievement levels in math, English/language arts, '''
                                 '''and science), there is a strong prohibition on the federal government using any '''
                                 '''of its authority to mandate or incentivize the use of particular standards. '''
                                 '''This prohibition not only applies to standards, but also assessments, '''
                                 '''curriculum, etc. The bill does note, however, that nothing in the law '''
                                 '''prohibits states from voluntarily entering into partnerships on standards.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 1.46 s, sys: 0 ns, total: 1.46 s
Wall time: 1.45 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>While states must maintain “challenging academic standards” (floor set as: at least three achievement levels in math, English/language arts, and science), there is a strong prohibition on the federal government using any of its authority to mandate or incentivize the use of particular standards. This prohibition not only applies to standards, but also assessments, curriculum, etc. The bill does note, however, that nothing in the law prohibits states from voluntarily entering into partnerships on standards.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The context provided states that the Every Student Succeeds Act (ESSA) prohibits the federal government from using its authority to mandate or incentivize the use of particular standards. This means that the federal government cannot mandate or incentivize the use of Common Core standards. <br><br>Therefore, the related mandates or prohibitions related to Common Core are:<br><br>* **No federal mandate:** The federal government cannot mandate the use of Common Core standards.<br>* **No federal incentivization:** The federal government cannot incentivize the use of Common Core standards. <br><end_of_turn></blockquote></font>

In [12]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''ESSA requires that a state’s accountability system must measure the performance of '''
                                 '''95 percent of students by looking at a variety of indicators. One of the indicators '''
                                 '''is “academic achievement as measured by proficiency on the annual assessments.” '''
                                 '''For this reason, in order to measure the overall achievement of 95 percent of students, '''
                                 '''95 percent must take the annual assessments.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 1.45 s, sys: 0 ns, total: 1.45 s
Wall time: 1.45 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments. <br><br><br>**Explanation:**<br><br>The answer is yes because the context explicitly states that ESSA requires states to test 95% of students. <br><end_of_turn></blockquote></font>

In [13]:
%%time
template = "{pre}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.'''
                            ''' Use the following context to answer the question below.''',
                         context='''There is clear language in the bill that no state shall be required to submit its standards '''
                                  '''to the federal government for review or approval. (Standards underlie the accountability system, '''
                                  '''which is part of the state Title I plan submitted to the Department.) '''
                                  '''Again, states must maintain challenging academic standards, but the law is '''
                                  '''very clear that states are not required to seek federal approval of their standards '''
                                  '''and can make changes to them without federal approval.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 416 ms, sys: 0 ns, total: 416 ms
Wall time: 412 ms


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act. Use the following context to answer the question below.<br><br>Context:<br>There is clear language in the bill that no state shall be required to submit its standards to the federal government for review or approval. (Standards underlie the accountability system, which is part of the state Title I plan submitted to the Department.) Again, states must maintain challenging academic standards, but the law is very clear that states are not required to seek federal approval of their standards and can make changes to them without federal approval.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>No, states are not required to submit their standards for approval by the U.S. Department of Education. <br><end_of_turn></blockquote></font>